In [1]:
# Import Required Libraries
import os
import pandas as pd
from datetime import datetime, timedelta
import random
import json
import findspark

findspark.init()

# PySpark imports
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import *
    from pyspark.sql.types import *
    pyspark_available = True
    print("PySpark is available!")
except ImportError:
    print("PySpark not found. Please install with: pip install pyspark")
    pyspark_available = False

if pyspark_available:
    # Create SparkSession with custom configuration
    spark = SparkSession.builder \
        .appName("DAT535-Project") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "1g") \
        .getOrCreate()
    
    # Set log level to reduce verbose output
    spark.sparkContext.setLogLevel("WARN")
    
    print("✓ SparkSession created successfully!")
    print(f"Spark Version: {spark.version}")
    print(f"Application Name: {spark.sparkContext.appName}")
    print(f"Master: {spark.sparkContext.master}")
    
    # Check available cores and memory
    print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")

    from pyspark.sql.functions import (
        col, lit, concat, current_timestamp, rand, when,
        lower, upper, date_format
    )
else:
    print("Cannot proceed without PySpark. Please install PySpark first.")

PySpark is available!


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 12:08:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ SparkSession created successfully!
Spark Version: 3.5.0
Application Name: DAT535-Project
Master: local[*]
Default Parallelism: 4


In [2]:
csv_path = 'data/ukprop_raw/ukprop_raw.csv'

In [5]:
ukprop_schema = StructType([
     StructField("transactionid", StringType(), True),
     StructField("price", StringType(), True),
     StructField("dateoftransfer", StringType(), True),
     StructField("postcode", StringType(), True),
     StructField("propertytype", StringType(), True),
     StructField("NUMBER_HEATED_ROOMS", StringType(), True),
     StructField("FLOOR_HEIGHT", StringType(), True),
     StructField("CURRENT_ENERGY_RATING", StringType(), True),
     StructField("oldnew", StringType(), True),
     StructField("duration", StringType(), True),
     StructField("paon", StringType(), True),
     StructField("saon", StringType(), True),
     StructField("street", StringType(), True),
     StructField("locality", StringType(), True),
     StructField("towncity", StringType(), True),
     StructField("district", StringType(), True),
     StructField("county15", StringType(), True),
     StructField("categorytype", StringType(), True),
     StructField("recordstatus", StringType(), True),
     StructField("tfarea",StringType(), True),
     StructField("numberrooms",StringType(), True)
 ])

In [6]:
raw_df = spark.read.option("header", "true").csv(csv_path)

columns_to_keep = [field.name for field in ukprop_schema.fields]

filtered_df = raw_df.select([c for c in columns_to_keep if c in raw_df.columns])
raw_df.printSchema()
filtered_df.printSchema()

filtered_df.show(10,truncate=False)
rdd = filtered_df.rdd


root
 |-- id: string (nullable = true)
 |-- transactionid: string (nullable = true)
 |-- oa11: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- price: string (nullable = true)
 |-- dateoftransfer: string (nullable = true)
 |-- propertytype: string (nullable = true)
 |-- oldnew: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- paon: string (nullable = true)
 |-- saon: string (nullable = true)
 |-- street: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- towncity: string (nullable = true)
 |-- district: string (nullable = true)
 |-- county15: string (nullable = true)
 |-- categorytype: string (nullable = true)
 |-- recordstatus: string (nullable = true)
 |-- year: string (nullable = true)
 |-- lsoa11: string (nullable = true)
 |-- msoa11: string (nullable = true)
 |-- laua: string (nullable = true)
 |-- lad11nm: string (nullable = true)
 |-- gor: string (nullable = true)
 |-- rgn11nm: string (nullable = true)
 |-- classt: str

25/11/10 12:08:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
 Schema: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county15, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
Expected: county15 but found: county
CSV file: file:///home/ubuntu/spark-notebooks/data/ukprop_raw/ukprop_raw.csv


## Construction of the sentences
### First part
description of the property :
propertytype, oldnew, duration, tfarea, numberrooms, 

### additional information
number_heated_rooms,floor_height,current_energy_rating

### Second part
the location :
PAON 	SAON 	Street 	 Locality 	 Town/City 	 District 	 County postcode

### Third part
The transaction : date_of_transfer, transaction id,price

### Fourth part
recording of the transaction : recordstatus, categorytype

In [7]:
# rows that are dropped often in final output: duration, city, record_status, category_type, property_type


In [8]:
def get_property_text(property_type):
    match property_type:
        case "D":
            return "Detached property"
        case "S":
            return "Semi-detached property"
        case "T":
            return "Terraced property"
        case "F":
            return random.choice(["Flat","Maisonette"])
        case "O":
            return "property"
        
def create_description_part(property_type,old_new,duration,tfarea,number_rooms):
    
    # Translations
    old_new_text = "newly built" if old_new == "N" else "old"
    duration_text = "freehold" if duration == "F" else "leasehold"
    room_text = f"{number_rooms} room" if number_rooms == 1 else f"{number_rooms} rooms"
    property_text= get_property_text(property_type)
    # possibles template
    templates = [
        f"A {old_new_text} {duration_text} {property_text.lower()} with {room_text} covering {tfarea} sqm.",
        f"An {old_new_text} {duration_text} {property_text.lower()} with {room_text} and {tfarea} sqm of living space.",
        f"This {old_new_text} {duration_text} {property_text.lower()} offers {room_text} over {tfarea} square meters."
    ]
    return random.choice(templates)

In [9]:
def create_additional_info_part(number_heated_rooms, floor_height, current_energy_rating):
    heated_text = f"{number_heated_rooms} heated room" if number_heated_rooms == 1 else f"{number_heated_rooms} heated rooms"
    
    floor_text = f"{floor_height} meters high"  # rounded to 1 decimal
    
    energy_text = f"energy rating {current_energy_rating}" if current_energy_rating else "energy rating unknown"
    
    templates = [
        f"The property features {heated_text},"+(f" with a ceiling height of {floor_text}," if floor_height and floor_height!="NA" else "") + f" and {energy_text}.",
        f"{heated_text}" + (f" and a ceiling {floor_text}" if floor_height and floor_height!="NA" else "") + f", currently rated {energy_text}.",
        f"This home offers {heated_text}"+  (f" and ceilings reaching {floor_text}" if floor_height and floor_height!="NA" else "") + f", accompanied by {energy_text}.",
        f"{heated_text},"+ (f"ceilings {floor_text}," if floor_height and floor_height!="NA" else "") + f" plus {energy_text}."
    ]
    
    return random.choice(templates)
print(create_additional_info_part("2",None,"D"))

The property features 2 heated rooms, and energy rating D.


In [10]:
def create_location_part(paon, saon, street, locality, city, district, county, postcode):
    paon = paon or ""
    saon = saon or ""
    street = street or ""
    locality = locality or ""
    city = city or ""
    district = district or ""
    county = county or ""
    postcode = postcode or ""

    # creating an address from available informations
    address_parts = [saon, paon, street]
    address = " ".join([p for p in address_parts if p]).strip()

    location_parts = [locality, city, district, county]
    location = ", ".join([l for l in location_parts if l]).strip(", ")

    templates = [f"Located at {address}, {location} ({postcode}) in the city of {city}.",
        f"Situated at {address}, {county} ({postcode}) in the city of {city}.",
        f"Found in the city of {city}, {county}, at {address} ({postcode}).",
        f"Nestled at {address}, within the city of {city or locality}, {county} ({postcode}).",
        f"Positioned at {address}, {street if street else locality}, in the city of {city or county} ({postcode})."
    ]

    if not address:
        templates = [
            f"Located in the city of {city or locality}, {county} ({postcode}).",
            f"Situated within the city of {city or district} area of {county} ({postcode}).",
            f"Found in the city of {city or locality}, {county} ({postcode})."
        ]

    return random.choice(templates)

In [12]:
def create_transaction_part(date_of_transfer, trans_id, price):
    templates = [
        f"The transaction took place on {date_of_transfer} with ID {trans_id} for £{price}.",
        f"Sold on {date_of_transfer}, transaction reference: {trans_id}, at a price of £{price}.",
        f"This property was transferred on {date_of_transfer} (Transaction ID: {trans_id}) for £{price}.",
        f"Date of transfer: {date_of_transfer}, Transaction ID: {trans_id}, sold for £{price}."
    ]
    
    return random.choice(templates)


In [13]:
def create_recording_part(record_status, category_type):

    record_status_map = {
        "A": "Addition",
        "C": "Change",
        "D": "Deletion"
    }
    status_text = record_status_map.get(record_status, "Unknown status")
    
  
    category_type_map = {
        "A": "Standard Price Paid entry",
        "B": "Additional Price Paid entry"
    }
    category_text = category_type_map.get(category_type, "General/Unknown category")
    
    templates = [
        f"The transaction has a record status of {status_text} and belongs to category: {category_text}.",
        f"Record status: {status_text}; Category type: {category_text}.",
        f"This transaction is classified as {category_text} with status {status_text}.",
        f"Transaction status: {status_text}, recorded under category: {category_text}."
    ]
    
    return random.choice(templates)


In [14]:

def create_sentence_from_line(line):
    description = create_description_part(line["propertytype"],line["oldnew"],line["duration"],line["tfarea"],line["numberrooms"])
    description += " "+create_additional_info_part(line["NUMBER_HEATED_ROOMS"],line["FLOOR_HEIGHT"],line["CURRENT_ENERGY_RATING"])
    description += " "+create_location_part(line["paon"],line["saon"],line["street"],line["locality"],line["towncity"],line["district"],line["county15"],line["postcode"])
    description+=" "+create_transaction_part(line["dateoftransfer"],line["transactionid"],line["price"])
    description+=" "+create_recording_part(line["recordstatus"],line["categorytype"])
    return Row(description=description)
rdd_unstructured = rdd.map(create_sentence_from_line)

df_text = spark.createDataFrame(rdd_unstructured)

df_text.show(5, truncate=False)

25/11/10 12:08:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
 Schema: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county15, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
Expected: county15 but found: county
CSV file: file:///home/ubuntu/spark-notebooks/data/ukprop_raw/ukprop_raw.csv
                                                                                

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                                                                                                                                                                                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------------------------------

25/11/10 12:08:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
 Schema: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county15, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
Expected: county15 but found: county
CSV file: file:///home/ubuntu/spark-notebooks/data/ukprop_raw/ukprop_raw.csv


In [15]:
df_text.select("description").coalesce(1).write.mode("overwrite").text("data/ukprop_unstructured.txt")

25/11/10 12:09:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
 Schema: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county15, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
Expected: county15 but found: county
CSV file: file:///home/ubuntu/spark-notebooks/data/ukprop_raw/ukprop_raw.csv
                                                                                

In [16]:
df_text.count()

25/11/10 12:10:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
 Schema: transactionid, postcode, price, dateoftransfer, propertytype, oldnew, duration, paon, saon, street, locality, towncity, district, county15, categorytype, recordstatus, tfarea, numberrooms, CURRENT_ENERGY_RATING, NUMBER_HEATED_ROOMS, FLOOR_HEIGHT
Expected: county15 but found: county
CSV file: file:///home/ubuntu/spark-notebooks/data/ukprop_raw/ukprop_raw.csv
                                                                                

5732838